## Décorateurs
Les décorateurs sont un concept relativement poussé en Python, qui utilise le fait que les fonctions sont des objets comme les autres.

En effet, la syntaxe
```python
def nom_de_fonction(<params>):
    <corps>
```
est l'équivalent théorique de cette affirmation (théorique, car la classe `function` ne peut pas être utilisée ainsi):
```python
nom_de_fonction = function(<params>, <corps>)
```

En effet, le nom donné à la fonction devient une façon de l'identifier.

Comme les fonctions sont des objets en Python, elle peuvent aussi être passées en argument à d'autres fonctions.

In [ ]:
def dire_bonjour(nom):
    print(f"Bonjour {nom}!")

def dire_bye(nom):
    print(f"Bye bye {nom}!")

def invoquer_fonction(fonction, argument): # La fonction reçoit une autre fonction en paramètre
    return fonction(argument) # La fonction reçue en paramètre est invoquée, et sa valeur de retour est retournée

invoquer_fonction(dire_bonjour, "Bob") # On peut saluer Bob
invoquer_fonction(dire_bye, "Gérard") # Ou dire au revoir à Gérard
print(invoquer_fonction(len, [1, 2, 3, 4, 5])) # Ou obtenir la longueur d'une liste!

### Créer une fonction qui modifie le comportement d'une autre fonction
Imaginons que j'aie une fonction `somme` et une fonction `produit`, qui prennent un itérable en paramètre et calculent la somme ou le produit de tous ses éléments.



Or, dans les deux cas, si l'itérable est vide, je souhaite afficher un message d'erreur dans la console, et retourner `None`.
Je pourrais ajouter l'extrait de code suivant au début de chaque fonction:
```python
if len(iterable) == 0:
    print("L'itérable est vide!")
    return None
```
Or, cela ferait de la duplication de code, ce qui est plus difficile à maintenir.
Je peux toutefois utiliser une méthode dynamique pour y arriver:

In [ ]:
from typing import Iterable

def somme(iterable: Iterable):
    somme = 0
    for val in iterable:
        somme += val
    return somme

def produit(iterable: Iterable):
    produit = 1
    for val in iterable:
        produit *= val
    return produit

def creer_verifier_vide(param_fonction): # On crée une fonction qui reçoit une fonction en argument

    def nouvelle_fonction(iterable: Iterable): # On crée une nouvelle fonction, qui viendra remplacer la fonction reçue en argument
        if len(iterable) == 0: # On fait la vérification sur l'argument `iterable`
            print("L'itérable est vide!")
            return None
        else:
            return param_fonction(iterable) # S'il n'est pas vide, on invoque la fonction reçue
    
    return nouvelle_fonction # On retourne notre nouvelle fonction

Le code ci-dessous crée les deux fonctions `somme` et `produit`, ainsi qu'une fonction `creer_verifier_vide` qui prend une fonction en argument, lui greffe la logique de vérification de la taille de l'itérable, et s'il n'est pas vide, invoque la fonction initiale avec l'itérable reçu comme argument, et retourne son résultat. Or, la fonction `creer_verifier_vide` retourne la fonction `nouvelle_fonction`, ce qui nous permet de l'assigner à une variable, et de l'utiliser.

Ainsi, si l'itérable n'est pas vide, la fonction `nouvelle_fonction` aura le même comportement que la fonction `param_fonction`. Par contre, si l'itérable est vide, elle affichera le message d'erreur dans la console, et retournera `None`.

On peut donc créer nos nouvelles fonctions:

In [ ]:
somme_verif = creer_verifier_vide(somme)
produit_verif = creer_verifier_vide(produit)

print(f"somme pas vide: {somme([1,2,3,4])}")
print(f"somme_verif pas vide: {somme_verif([1,2,3,4])}")
print(f"somme vide: {somme([])}")
print(f"somme_verif vide: {somme_verif([])}")
print("-"*30)
print(f"produit pas vide: {produit([1,2,3,4])}")
print(f"produit_verif pas vide: {produit_verif([1,2,3,4])}")
print(f"produit vide: {produit([])}")
print(f"produit_verif vide: {produit_verif([])}")

On a maintenant une version des fonctions `somme` et `produit` avec la vérification voulue!
Et comme on a englobé la fonctionnalité dans une autre fonction, `creer_verifier_vide`, on pourrait ajouter la même vérification à plusieurs autres fonctions très facilement!

In [ ]:
def concatener(iterable: Iterable[str]):
    chaine = ""
    for val in iterable:
        chaine = f"{chaine}{val}"
    return chaine

concatener_verif = creer_verifier_vide(concatener)

print(f"concatener pas vide: {repr(concatener(['a','l','l','o']))}")
print(f"concatener_verif pas vide: {repr(concatener_verif(['a','l','l','o']))}")
print(f"concatener vide: {repr(concatener([]))}")
print(f"concatener_verif vide: {repr(concatener_verif([]))}")

Maintenant, si on ne souhaite pas garder la version sans la vérification, on peut simplement assigner la nouvelle version à l'ancien nom!

In [ ]:
somme = creer_verifier_vide(somme)

print(f"somme remplacée pas vide: {somme([1,2,3,4])}")
print(f"somme remplacée vide: {somme([])}")

#### Décorateur et la syntaxe `@`
Notre fonction `creer_verifier_vide` est un décorateur: elle **décore** une autre fonction, en lui greffant des fonctionnalités avant et/ou après.

Maintenant, il existe une syntaxe équivalente à l'assignation
```python
def func(<args>):
    <corps>

func = decorateur(func)
```

C'est la syntaxe avec le `@`:
```python
@decorateur
def func(<args>):
    <corps>
```

In [ ]:
@creer_verifier_vide # On utilise la syntaxe de décorateur avec `@`
def creer_tuple(iterable: Iterable):
    liste = []
    for val in iterable:
        liste.append(val)
    return tuple(liste)

print(f"creer_tuple remplacé pas vide: {creer_tuple([1,2,3,4])}")
print(f"creer_tuple remplacé vide: {creer_tuple([])}")

### Décorateur avec des paramètres
Pour créer un décorateur prenant des paramètres, on utilise trois niveaux de fonctions imbriquées:
1. Prend les arguments que l'on veut en paramètres, et retourne un **décorateur**
2. Prend une fonction en paramètre, et retourne une **nouvelle fonction**
3. Décore une fonction, et fait office de nouvelle fonction

In [ ]:
def creer_verifier_vide_message(message): # On crée une fonction qui capture nos paramètres
    def decorateur(param_fonction):
        def nouvelle_fonction(iterable: Iterable): # On crée une nouvelle fonction, qui viendra remplacer la fonction reçue en argument
            if len(iterable) == 0: # On fait la vérification sur l'argument `iterable`
                print(f"Voici mon message: <{message}>") # On affiche le message reçu en argument au premier niveau
                return None
            else:
                return param_fonction(iterable) # S'il n'est pas vide, on invoque la fonction reçue
        
        return nouvelle_fonction # On retourne notre nouvelle fonction
    return decorateur # On retourne le décorateur

@creer_verifier_vide_message("La taille de l'itérable est de 0!")
def creer_set(iterable: Iterable):
    mon_set = set()
    for val in iterable:
        mon_set.add(val)

    return mon_set

print(f"creer_set remplacé pas vide: {creer_set([1,2,3,4])}")
print(f"creer_set remplacé vide: {creer_set([])}")

### Conserver les attributs avec `functools.wraps`
L'utilisation d'un décorateur remplace certains attributs de la fonction. Parfois, on aimerait garder ces informations.

In [ ]:
def fonction_au_nom_genial(iterable: Iterable):
    """La documentation de ma fonction au nom génial est ici!"""
    print("La longueur de mon itérable est:", len(iterable))
    return len(iterable)

fonction_au_nom_genial_2 = creer_verifier_vide(fonction_au_nom_genial)

print(fonction_au_nom_genial.__name__)
print(fonction_au_nom_genial.__doc__)
print("-"*60)
print(fonction_au_nom_genial_2.__name__)
print(fonction_au_nom_genial_2.__doc__)


On remarque que le nom et la documentation, par exemple, sont perdues. Le phénomène est le même si on utilise directement la syntaxe avec `@`.

In [ ]:
@creer_verifier_vide
def fonction_au_nom_magnifique(iterable: Iterable):
    """La documentation de ma fonction au nom magnifique est là!"""
    print("La longueur de mon itérable est:", len(iterable))
    return len(iterable)

print(fonction_au_nom_magnifique.__name__)
print(fonction_au_nom_magnifique.__doc__)

On peut remédier à la situation en ajoutant un élément dans la définition de notre décorateur: on peut identifier notre `nouvelle_fonction` comme un enrobage de la fonction initiale, à l'aide... d'un décorateur!

En effet, le décorateur `functools.wraps` prend en paramètre la fonction d'origine, et permet de facilement transférer les informations de la fonction initiale. 

In [ ]:
from functools import wraps

def creer_verifier_vide(param_fonction): # On crée une fonction qui reçoit une fonction en argument

    @wraps(param_fonction) # Ici, on précise que la fonction enrobe la fonction précédente
    def nouvelle_fonction(iterable: Iterable): # On crée une nouvelle fonction, qui viendra remplacer la fonction reçue en argument
        if len(iterable) == 0: # On fait la vérification sur l'argument `iterable`
            print("L'itérable est vide!")
            return None
        else:
            return param_fonction(iterable) # S'il n'est pas vide, on invoque la fonction reçue
    
    return nouvelle_fonction # On retourne notre nouvelle fonction

@creer_verifier_vide
def fonction_au_nom_parfait(iterable: Iterable):
    """La documentation de ma fonction au nom parfait est invincible!"""
    print("La longueur de mon itérable est:", len(iterable))
    return len(iterable)

print(fonction_au_nom_parfait.__name__) # Les valeurs affichées seront les bonnes
print(fonction_au_nom_parfait.__doc__)